# 7. Preparing Excel file

In this script, we will prepare a pivoted Excel file with the results for key cities.
This will be used by other reporters and editors to find interesting patterns in the data.

In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
# Loads city data to re-add extra inormation
cities = gpd.read_file("../input/GHS_STAT_UCDB2015MT_GLOBE_R2019A/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg")

In [3]:
#  Keep only the desired columns
cities = cities[['ID_HDC_G0', 'UC_NM_MN', 'UC_NM_LST', 'CTR_MN_NM', 'CTR_MN_ISO',
                 'AREA', 'P15', 'E_KG_NM_LST', 'E_BM_NM_LST']]

In [4]:
# Also load the extra regional information
info = pd.read_csv("../input/country_classifications/country-and-continent-codes-list.csv", sep=";")

In [5]:
# Keeps only the desired columns
info = info[['continent', 'region', 'subregion', 'ISO-alpha3 code']]

In [6]:
# Gets hexbin data
hexbins = pd.read_csv("../output/population-by-tree-coverage-bin.csv")

In [7]:
hexbins.head()

,city_id,tr_pct_bin,pop_ft,pop_ft_pct,UC_NM_MN,CTR_MN_NM,CTR_MN_ISO,continent,INCM_CMI
0,10.0,"[0.0, 0.02)",535863.397407,0.113691,San Jose,United States,USA,Americas,HIC
1,10.0,"[0.02, 0.04)",696202.730230,0.147709,San Jose,United States,USA,Americas,HIC
2,10.0,"[0.04, 0.06)",757926.919928,0.160805,San Jose,United States,USA,Americas,HIC
3,10.0,"[0.06, 0.08)",617455.151767,0.131002,San Jose,United States,USA,Americas,HIC
4,10.0,"[0.08, 0.1)",467465.030952,0.099179,San Jose,United States,USA,Americas,HIC


In [8]:
# Adds information back to the hexbins
hexbins = hexbins.merge(cities, left_on='city_id', right_on='ID_HDC_G0', suffixes=('','_y'))\
       .merge(info, left_on='CTR_MN_ISO', right_on='ISO-alpha3 code', suffixes=('','_y'))
hexbins = hexbins.drop(columns=[col for col in hexbins.columns if '_y' in col ])

In [9]:
# Creates a pivot table
distributions = hexbins.pivot(index='city_id', columns='tr_pct_bin', values='pop_ft_pct').reset_index().fillna(0).round(3)

In [10]:
# Rename columns so it's more friendly
def interval_to_string(interval):
    start = float(interval.split(',')[0][1:]) * 100
    end = float(interval.split(',')[1][:-1]) * 100
    return f"{round(start)}% – {round(end)}%"

# Apply the function to the column
distributions = distributions.rename(columns={ colname: interval_to_string(colname) for colname in distributions.columns[1:] })


In [11]:
# Gets the most common bin
# All rows (:)
# Starting from the column with name '0% – 2%' ('0% – 2%':)
distributions['most_common'] = distributions.loc[:, '0% – 2%':].idxmax(axis=1)

In [12]:
# Gets the lower bound of the most common place by splitting the resulting string
# and then getting the first element. Removes the % sign and casts as int.
distributions['most_common_lower_bound'] = distributions.most_common.str.split('–')\
                                              .apply(lambda x: int(x[0].strip().split('%')[0]))

In [13]:
# Joins back the infomrative columns
distributions = distributions.merge(hexbins, on='city_id').drop_duplicates(subset='city_id')\
    .drop(['tr_pct_bin', 'pop_ft', 'pop_ft_pct', 'ID_HDC_G0'], axis=1)

In [14]:
# Renames and orders so it's more friendly
distributions.columns

Index(['city_id', '0% – 2%', '2% – 4%', '4% – 6%', '6% – 8%', '8% – 10%',
       '10% – 12%', '12% – 14%', '14% – 16%', '16% – 18%', '18% – 20%',
       '20% – 22%', '22% – 24%', '24% – 26%', '26% – 28%', '28% – 30%',
       '30% – 32%', '32% – 34%', '34% – 36%', '36% – 38%', '38% – 40%',
       '40% – 42%', '42% – 44%', '44% – 46%', '46% – 48%', '48% – 50%',
       '50% – 52%', '52% – 54%', '54% – 56%', '56% – 58%', '58% – 60%',
       '60% – 62%', '62% – 64%', '64% – 66%', '66% – 68%', '68% – 70%',
       '70% – 72%', '72% – 74%', '74% – 76%', '76% – 78%', '78% – 80%',
       '80% – 82%', '82% – 84%', '84% – 86%', '86% – 88%', '88% – 90%',
       '90% – 92%', '92% – 94%', '94% – 96%', '96% – 98%', '98% – 100%',
       'most_common', 'most_common_lower_bound', 'UC_NM_MN', 'CTR_MN_NM',
       'CTR_MN_ISO', 'continent', 'INCM_CMI', 'UC_NM_LST', 'AREA', 'P15',
       'E_KG_NM_LST', 'E_BM_NM_LST', 'region', 'subregion', 'ISO-alpha3 code'],
      dtype='object')

In [15]:
distributions = distributions.rename(columns={
    "city_id": "City unique id",
    "most_common": "Most people live in areas with _% tree coverage",
    "most_common_lower_bound": "Most people live in areas with _% tree coverage (lower bound, use for sorting)",
    "UC_NM_MN": "Main city name",
    "UC_NM_LST": "Cities in area",
    "CTR_MN_NM": "Main country name",
    "CTR_MN_ISO": "Main country ISO3 code",
    "continent": "Continent",
    "region": "Region",
    "subregion": "Subregion",
    "INCM_CMI": "Income level of country",
    "AREA": "Area in square kms",
    "P15": "Population estimate (2015, satellite model)",
    "E_KG_NM_LST": "List of climates (Köppen-Geiger)",
    "E_BM_NM_LST": "List of biomes (WWF)"
})

In [16]:
# Reorder columns
distributions = distributions[
    [
           'City unique id',
           'Main city name', 'Cities in area', 'Main country name', 'Main country ISO3 code',
           'Continent','Region', 'Subregion', 
           'Income level of country',
           'Area in square kms', 'Population estimate (2015, satellite model)',
           'List of climates (Köppen-Geiger)', 'List of biomes (WWF)',
            'Most people live in areas with _% tree coverage',
           'Most people live in areas with _% tree coverage (lower bound, use for sorting)',
            '0% – 2%', '2% – 4%', '4% – 6%', '6% – 8%',
           '8% – 10%', '10% – 12%', '12% – 14%', '14% – 16%', '16% – 18%',
           '18% – 20%', '20% – 22%', '22% – 24%', '24% – 26%', '26% – 28%',
           '28% – 30%', '30% – 32%', '32% – 34%', '34% – 36%', '36% – 38%',
           '38% – 40%', '40% – 42%', '42% – 44%', '44% – 46%', '46% – 48%',
           '48% – 50%', '50% – 52%', '52% – 54%', '54% – 56%', '56% – 58%',
           '58% – 60%', '60% – 62%', '62% – 64%', '64% – 66%', '66% – 68%',
           '68% – 70%', '70% – 72%', '72% – 74%', '74% – 76%', '76% – 78%',
           '78% – 80%', '80% – 82%', '82% – 84%', '84% – 86%', '86% – 88%',
           '88% – 90%', '90% – 92%', '92% – 94%', '94% – 96%', '96% – 98%',
           '98% – 100%'
    ]
]

In [17]:
# Saves as Excel
distributions.to_excel("../output/Population distribution in major cities.xlsx")

In [23]:
distributions.head()

,City unique id,Main city name,Cities in area,Main country name,Main country ISO3 code,Continent,Region,Subregion,Income level of country,Area in square kms,...,80% – 82%,82% – 84%,84% – 86%,86% – 88%,88% – 90%,90% – 92%,92% – 94%,94% – 96%,96% – 98%,98% – 100%
0,10.0,San Jose,San Jose; San Francisco; Oakland; Fremont; Sun...,United States,USA,Americas,Northern America,Northern America,HIC,1717.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,79.0,Phoenix,Phoenix; Glendale; Scottsdale; Gilbert; Peoria...,United States,USA,Americas,Northern America,Northern America,HIC,2304.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,143.0,Ciudad Juárez,Ciudad Juárez; El Paso,Mexico,MEX,Americas,Central America,Central America,UMIC,534.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57,183.0,Anchorage,Anchorage,United States,USA,Americas,Northern America,Northern America,HIC,60.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,288.0,Guatemala City,Guatemala City; Villa Nueva City; Mixco,Guatemala,GTM,Americas,Central America,Central America,LMIC,411.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Additionally, I will sabe basic information about all cities, despite not having the population distribution for them.

In [18]:
# Processed data
df = pd.read_csv("../output/earth_engine_exports/TreeCoverAnalysis-Cities-27-08-24-08_32.csv")

In [19]:
# Joins it with the original city data
df = df.merge(cities, on='ID_HDC_G0')

In [20]:
# Joins with the extra information
df = df.merge(info, left_on='CTR_MN_ISO', right_on='ISO-alpha3 code')

In [21]:
# Renames columns
to_rename = {
    "ID_HDC_G0": "City id on GHSL",
    "UC_NM_MN": "Urban center main city name",
    "UC_NM_LST": "Cities in uban center",
    "CTR_MN_NM": "Name of main city country",
    "CTR_MN_ISO": "ISO code of main city country",
    "lon": "Longitude",
    "lat": "Latitude",
    "continent": "Continent",
    "region": "Region",
    "subregion": "Subregion",
    "AREA": "City area, square km",
    "P15": "Population (2015 satellite estimate, preferable)",
    "pop_ft": "Population within city, computed by us (2020 satellite estimate)",
    "tree_area": "Area covered by trees, in sqaure m",
    "tree_pct": "Percentage covered by trees",

}

df = df.rename(columns=to_rename)
df = df[to_rename.values()]
df.head()

,City id on GHSL,Urban center main city name,Cities in uban center,Name of main city country,ISO code of main city country,Longitude,Latitude,Continent,Region,Subregion,"City area, square km","Population (2015 satellite estimate, preferable)","Population within city, computed by us (2020 satellite estimate)","Area covered by trees, in sqaure m",Percentage covered by trees
0,5959.0,Herat,Herat,Afghanistan,AFG,62.209328,34.345746,Asia,Southern Asia,Southern Asia,60.0,844574.564331,1.061821e+06,990417.400172,0.016543
1,5964.0,Guzarah,Guzarah,Afghanistan,AFG,62.218433,34.218091,Asia,Southern Asia,Southern Asia,16.0,160204.260864,1.383606e+05,215990.452808,0.013533
2,5968.0,Shindand,Shindand; Sabzawar,Afghanistan,AFG,62.159005,33.305511,Asia,Southern Asia,Southern Asia,12.0,147553.403870,1.618679e+05,218078.836547,0.018214
3,5970.0,Qala i Naw,Qala i Naw,Afghanistan,AFG,63.129197,34.987879,Asia,Southern Asia,Southern Asia,3.0,79809.722656,6.510151e+04,35443.368529,0.011840
4,5973.0,Farah,Farah,Afghanistan,AFG,62.105844,32.373878,Asia,Southern Asia,Southern Asia,16.0,122843.460327,1.148601e+05,304222.823549,0.019058


In [22]:
df.to_excel("../output/Basic info - all cities.xlsx")